In [23]:
import os
import pandas as pd
import praw
import time
import pickle
# from dotenv import load_dotenv
from plotnine import *
from praw.models import MoreComments

In [3]:
username = "SamTheIceCreamMan"
password = "Samernator1"
id = "U9EagV0j3qUSbw"
secret = "Su_o5xTzvs4RKygGOBakQFutxw49-A"
user_agent = "scrapermaster"

In [4]:
reddit = praw.Reddit(client_id=id,
                     client_secret=secret,
                     password=password, user_agent=user_agent,
                     username=username)

In [5]:
sub_limit = 150
n_top = 50
n_old = 1000

In [9]:
subs = ["wallstreetbets", "stocks", "investing", "CyptoCurrency", "StockMarket", "pennystocks","WallStreetbetsELITE",]

In [10]:
def get_post_info(post, sub_name):
    info = {
        'sub': sub_name,
        'post_id': post.id,
        'title': post.title,
        'comments': post.num_comments,
        'score': post.score,
        'created_utc': post.created_utc
    }

    return info

def get_comment_info(comment, sort):    
    info = {
        #'sub': comment.subreddit.title,
        'post_id': comment.submission.id,
        'comment_id': comment.id,
        'created_utc': comment.created_utc,
        'score': comment.score,
        'top_level': comment.parent_id.startswith('t3_'),
        'sort': sort}

    return info

def get_comments(post, sort, n):
    post.comment_sort = sort
    post.comment_limit = n
    #post.comments.replace_more(limit=0)
    
    return [get_comment_info(c, sort) for c in post.comments.list() if not isinstance(c, MoreComments)]


def save_cmnt_df(cmnt_list, sub_name, n, perc):
    df_cmnt = pd.DataFrame(cmnt_list)
    df = df_cmnt.merge(pd.DataFrame(post_list), on='post_id')
    df.columns = ['post_id', 'comment_id', 'comment_created_utc', 'comment_score', 'top_level',
                  'sort', 'sub', 'post_title', 'comments_n', 'post_score', 'post_created_utc']
    df = df.sort_values(['post_id', 'comment_created_utc', 'sort']).reset_index(drop=True)
    df['in_old'] = df['comment_id'].isin(df.loc[df['sort']=='old', 'comment_id'].unique())
    df = df.drop_duplicates(subset=['post_id', 'comment_id'], keep='last')
    df['t_delta_min'] = (df['comment_created_utc'] - df['post_created_utc'])/60
    df['comment_score_pct'] = df.groupby(['post_id'], as_index=False)['comment_score'].transform(lambda x: x/x.sum())
    df['nth'] = df.groupby('post_id')['comment_id'].transform(lambda x: range(len(x)))
    df.to_csv(f'./data/n-{n}_perc-{perc}_{sub_name}.csv', index=False)

In [ ]:
for sub_name in subs:
    print(sub_name)
    post_list, top_list, old_list = [], [], []
    sub = reddit.subreddit(sub_name)
    
    posts = sub.top(limit=sub_limit)
    #posts = [p for p in posts if p.num_comments > n_top*10]
    for post in posts:
        post_list.append(get_post_info(post, sub_name))
        top_list.extend(get_comments(post, sort='top', n=n_top))
    
    posts = sub.top(limit=sub_limit)
    for post in posts:
        old_list.extend(get_comments(post, sort='old', n=n_old))
    
    pickle.dump(post_list, open(f'./data/posts_{sub_name}.pickle', 'wb'))
    pickle.dump(top_list, open(f'./data/top_{sub_name}.pickle', 'wb'))
    pickle.dump(old_list, open(f'./data/old_{sub_name}.pickle', 'wb'))
    #save_cmnt_df(post_list, cmnt_list, sub_name, n, perc)

wallstreetbets
stocks
investing
CyptoCurrency
StockMarket
pennystocks


In [20]:

def interpolate_n(x):
    mean_rate = x.iloc[int(len(x)*0.75):-1]['rate'].mean()
    x.loc[x['in_old']==False, 'nth'] = x.loc[x['in_old']==False, 't_delta_min']*mean_rate
    
    return x

def in_oldest(x):
    max_time = x.loc[x['sort']=='old', 'dt_min_post'].max()
    x['in_old'] = x['dt_min_post'] <= max_time
    
    return x

def lists_to_df(post_list, top_list, old_list, n_top):
    df = pd.DataFrame(top_list)
    df = df.sort_values(['post_id', 'score'], ascending=False).groupby('post_id').head(n_top)
    df = df.append(pd.DataFrame(old_list)).merge(pd.DataFrame(post_list), on='post_id')
    df.columns = ['post_id', 'comment_id', 'comment_created_utc', 'comment_score', 'top_level',
                 'sort', 'sub', 'post_title', 'comments_n', 'post_score', 'post_created_utc']
    
    ### calculate time since post and if the comment is within the top n oldest
    df['dt_min_post'] = ((df['comment_created_utc'] - df['post_created_utc'])/60).clip(0, None)
    df['dt_min_cmnt'] = df.groupby('post_id', as_index=False)['comment_created_utc'].transform(lambda x: (x-x.min())/60).clip(0, None)
    df = df.groupby('post_id', as_index=False).apply(in_oldest)
    
    ### sort by sort, creation time; drop dupes to retain only old not in top
    df = df.sort_values(['post_id', 'sort', 'comment_created_utc']).reset_index(drop=True)
    df = df.drop_duplicates(subset=['post_id', 'comment_id'], keep='last') # df['sort'].unique() = ['old', 'top']
    
    ### sort by score, assign rank
    df = df.sort_values(['post_id', 'comment_score'], ascending=False).reset_index(drop=True)
    df['rank'] = df.groupby('post_id', as_index=False)['comment_id'].transform(lambda x: [i+1 for i in range(len(x))])
    df.loc[df['sort']=='old', 'rank'] = None
    
    ### sort only by creation time, assign nth; if not in oldest, replace with predicted n
    df = df.sort_values(['post_id', 'comment_created_utc']).reset_index(drop=True)
    df['nth'] = df.groupby('post_id', as_index=False)['comment_id'].transform(lambda x: [i+1 for i in range(len(x))])
    #df['rate'] = df['nth'] / (df['t_delta_min']+0.5)
    df.loc[df['in_old']==False, 'nth'] = None
    #df = df.groupby('post_id', as_index=False).apply(interpolate_n)
    #df = df.drop(columns=['rate'])
    
    ### calculate score dominance and 
    df['comment_score_pct'] = df.groupby(['post_id'], as_index=False)['comment_score'].transform(lambda x: x/x.sum())
    df['norm_dom'] = df.groupby(['post_id'], as_index=False)['comment_score'].transform(lambda x: (x/x.sum())/(1/len(x)))
    #df['nth_perc'] = (df['nth'] / df['comments_n']).clip(None, 1)
    
    ### label cleanup, drop unneeded
    df.loc[df['sort']=='old', 'sort'] = 'oldest ~500 comments/post'
    df.loc[df['sort']=='top', 'sort'] = f'top {n_top} comments/post'
                                                                                         
    return df


def unpickle(sub_name):
    post_list = pickle.load(open(f'./data/posts_{sub_name}.pickle', 'rb'))
    top_list = pickle.load(open(f'./data/top_{sub_name}.pickle', 'rb'))
    old_list = pickle.load(open(f'./data/old_{sub_name}.pickle', 'rb'))
    
    return post_list, top_list, old_list


df_list = [lists_to_df(*unpickle(sub_name), n_top=10) for sub_name in subs]
df = pd.concat(df_list)
df = df.loc[(df['comments_n'] > 500)]
df = df.loc[(df['dt_min_post'] <= 1440)]
df = df.groupby('post_id').filter(lambda x: len(x)>250)

df.tail()

FileNotFoundError: [Errno 2] No such file or directory: './data/posts_wallstreetbets.pickle'